# Çekişmeli Üretici Ağlar (Generative Adversarial Networks) (GANs)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.utils as utils
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
device

device(type='cpu')

**İmage Generation**:MNIST veri seti \
GAN : Birbirine karşı çalışan iki yapay sinir ağı

In [4]:
batch_size = 12 # Min batch boyutu
image_size = 28 * 28 # Görüntü boyutu
transform = transforms.Compose([
    transforms.ToTensor(), # Görüntüleri Tensore çevir
    transforms.Normalize((0.5,),(0.5,)) # normalizasyon -> -1 ile +1 arasında sınırla
])

In [5]:
# MNIST DATASET
dataset = datasets.MNIST(root = "./mnist_data" , train = True , transform=transform , download = False)

In [6]:
data_loader = DataLoader(dataset , batch_size = batch_size , shuffle = True)

In [8]:
data_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist_data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

**Discriminator (Ayırt Edici)** \
Generatorin üretmiş olduğu görüntüleri gerçek mi sahte mi olduğunu anlamaya çalışacak

In [11]:
class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator , self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size,1024) , # input : image size , 1024 : nöron sayısı yani bu layer in outpu'u
            nn.LeakyReLU(0.2), # Aktivasyon fonk. 0.2 ' lik eğim
            nn.Linear(1024 , 512), # 1024 ten 512 düğüme
            nn.LeakyReLU(0.2),
            nn.Linear(512 , 256), # 512 den 256 ya
            nn.LeakyReLU(0.2),
            nn.Linear(256,1), # Output Layer
            nn.Sigmoid() # Çıktıyı 0-1 aralığına getirir
        )
    def forward(self,img):
        return self.model(img.view(-1,image_size)) # Görüntüyü düzleştirerek modele ver .

**Generator** \
Görüntü Oluşturma

In [17]:
class Generator(nn.Module):
    
    def __init__(self,z_dim):
        super(Generator , self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim , 256), # Girişten 256 düğüme tam bağlı katman
            nn.ReLU(),
            nn.Linear(256 , 512) ,
            nn.ReLU(),
            nn.Linear(512 , 1024),
            nn.ReLU(),
            nn.Linear(1024 , image_size), # 1024 ten (28 x 28) 784'e çevrim
            nn.Tanh() # Çıktı aktivasyon fonk.
        )

    def forward(self,x):
        return self.model(x).view(-1 , 1 , 28 , 28) # Çıktıyı 28 x 28 lik görüntüye çevirir 

**GAN Eğitimi**

In [18]:
# Hyperparameters
learning_rate = 0.0002
z_dim = 100 # Rastgele gürültü vektör boyutu (noise görüntüsü)
epochs = 20 # Eğitim döngü sayısı

# Model Başlatma : Generator ve discriminator tanımlama
generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)

# Kayıp fonksiyonu ve optimizasyon algoritmalarının tanımlanması
criterion = nn.BCELoss() # Binary cross entropy
g_optimizer = optim.Adam(generator.parameters() , lr = learning_rate , betas = (0.5 , 0.999)) # Generator Optimizer
d_optimizer = optim.Adam(discriminator.parameters() , lr = learning_rate , betas = (0.5,0.999)) # Discriminator Optimizer

# Eğitim Döngüsü Başlatma
for epoch in range(epochs):
    # Görüntülerin yüklenmesi
    for i , (real_imgs , _) in enumerate(data_loader):
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0) # Mevcut batch boyutunu al
        real_labels = torch.ones(batch_size , 1).to(device) # Gerçek görüntüleri 1 olarak etiketle
        fake_labels = torch.zeros(batch_size , 1).to(device) # Sahte görüntüleri 0 olarak etiketle

        # Discriminator Eğitimi
        z = torch.randn(batch_size , z_dim).to(device) # Rastgele gürültü üret
        fake_imgs = generator(z) # Generator ile sahte görüntü oluştur
        real_loss = criterion(discriminator(real_imgs),real_labels) # Gerçek görüntü kaybı
        fake_loss = criterion(discriminator(fake_imgs.detach()) ,fake_labels) # Sahte görüntü kaybı
        d_loss = real_loss + fake_loss # Toplam discriminator kaybı
    
        d_optimizer.zero_grad() # Gradyanları sıfırla
        d_loss.backward() # Geriye yayılım
        d_optimizer.step() # Parametreleri güncelle
    
        # Generator Eğitimi
        g_loss = criterion(discriminator(fake_imgs) , real_labels) # Generator kaybı
        g_optimizer.zero_grad() # Gradyanları sıfırla
        g_loss.backward() # Geriye yayılım
        g_optimizer.step() # Parametreleri güncelle

    print(f"Epoch : {epoch+1} / {epochs}  -  DLoss : {d_loss.item():.3f}  -  GLoss : {g_loss.item():.3f}")

Epoch : 1 / 20  -  DLoss : 0.256  -  GLoss : 2.760
Epoch : 2 / 20  -  DLoss : 0.405  -  GLoss : 3.622
Epoch : 3 / 20  -  DLoss : 0.113  -  GLoss : 3.464
Epoch : 4 / 20  -  DLoss : 1.161  -  GLoss : 1.436
Epoch : 5 / 20  -  DLoss : 1.073  -  GLoss : 1.589
Epoch : 6 / 20  -  DLoss : 1.024  -  GLoss : 1.354
Epoch : 7 / 20  -  DLoss : 1.050  -  GLoss : 1.546
Epoch : 8 / 20  -  DLoss : 1.194  -  GLoss : 1.547
Epoch : 9 / 20  -  DLoss : 1.522  -  GLoss : 1.398
Epoch : 10 / 20  -  DLoss : 1.061  -  GLoss : 1.191
Epoch : 11 / 20  -  DLoss : 1.215  -  GLoss : 1.209
Epoch : 12 / 20  -  DLoss : 1.016  -  GLoss : 1.283
Epoch : 13 / 20  -  DLoss : 1.023  -  GLoss : 1.308
Epoch : 14 / 20  -  DLoss : 0.792  -  GLoss : 1.279
Epoch : 15 / 20  -  DLoss : 0.878  -  GLoss : 1.275
Epoch : 16 / 20  -  DLoss : 1.083  -  GLoss : 1.289
Epoch : 17 / 20  -  DLoss : 1.150  -  GLoss : 1.237
Epoch : 18 / 20  -  DLoss : 0.953  -  GLoss : 1.735
Epoch : 19 / 20  -  DLoss : 1.017  -  GLoss : 1.263
Epoch : 20 / 20  -  D

**Modelin Test Edilmesi**

In [ ]:
# Rastgele gürültü ile görüntü oluşturma
with torch.no_grad():
    z = torch.randn(16,z_dim).to(device) # 16 adet rastgele görüntü oluştur
    sample_imgs = generator(z).cpu() # Generator ile sahte görüntü oluşturma
    grid = np.transpose(utils.make_grid(sample_imgs , nrow = 4 , normalize = True ) , (1,2,0))
    plt.imshow(grid)
    plt.show()